# 자동미분(Autograd 이해)

In [5]:
# 경사하강법 코드를 보고 있으면 requires_grad = True, backward() 등이 나온다. 
# 이는 파이토치에서 제공하는 자동 미분 기능을 수행하고 있는 것이다. 

In [6]:
import torch
# 임의의 스칼라 텐서 w 선언
# requires_grad = True는 이 텐서에 대한 기울기를 저장하겠다는 의미. 이렇게 하면 'grad'속성에 w에 대한 미분값이 저장됌
w = torch.tensor(2.0, requires_grad = True)

In [7]:
y = w**2
z = 2*y + 5

In [8]:
# backward()를 호출하면 해당 수식에서 requires_grad = True로 설정한 텐서들에 대한 기울기를 계산한다. 
z.backward()

In [11]:
print(f'수식을 w로 미분한 값 : {w.grad}')

수식을 w로 미분한 값 : 8.0


# 선형 회귀 구현 

In [3]:
x_train = torch.FloatTensor([[73,80,75],
                             [93,88,93],
                             [89,91,90],
                             [96,98,100],
                             [73,66,70]])

In [2]:
y_train = torch.FloatTensor([[152],[185],[180],[196],[142]])

In [13]:
# 모델 초기화 
W = torch.zeros((3,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = torch.optim.SGD([W,b],lr = 1e-5)     ## learning rate에 따라 발산할 수 있다. lr = 0.01만 돼도 발산한다. 
nb_epochs = 100

for epoch in range(nb_epochs) : 
    hypothesis = x_train.matmul(W) + b
    cost = torch.mean((hypothesis - y_train)**2)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {}  Cost : {:.6f}'.format(epoch+1, nb_epochs,
                                                               hypothesis.squeeze().detach(),
                                                               cost.item()
                                                               ))

Epoch    1/100 hypothesis: tensor([0., 0., 0., 0., 0.])  Cost : 29661.800781
Epoch    2/100 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605])  Cost : 9298.520508
Epoch    3/100 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821])  Cost : 2915.712891
Epoch    4/100 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097])  Cost : 915.040527
Epoch    5/100 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307])  Cost : 287.936005
Epoch    6/100 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891])  Cost : 91.371010
Epoch    7/100 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812])  Cost : 29.758139
Epoch    8/100 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805])  Cost : 10.445305
Epoch    9/100 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440])  Cost : 4.391228
Epoch   10/100 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396])  Cost

In [14]:
W.grad

tensor([[-5.5102],
        [ 6.0929],
        [-0.5702]])

In [15]:
W

tensor([[0.6735],
        [0.6610],
        [0.6762]], requires_grad=True)

### pytorch가 제공하는 nn.Module을 사용한 모델 구현
### 대부분의 파이토치 구현이 다음 과정을 따른다. 

In [16]:
import torch.nn as nn

In [17]:
class MultivariateLinearRegressionModel(nn.Module) :
    def __init__(self) : # 여기서 모델의 구조를 정의하는 생성자를 정의한다. 
        super().__init__()   # (above) 이는 객체가 갖는 속성값을 초기화 하는 역할로, 객체가 생성될때 자동으로 호출됌.
        self.linear = nn.Linear(3,1)  # 입력차원 3, 출력차원 1 
        
    def forward(self, x) :      # forward 함수는 모델이 학습데이터를 입력받아서 forward연산 (예측)을 진행하는 함수
        return self.linear(x)        # 모델 객체를 데이터와 함께 호출하면 자동으로 실행됌. 

In [18]:
model = MultivariateLinearRegressionModel()

In [21]:
# 모델에 저장되어 있는 파라미터 확인 
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.3791, -0.1642, -0.5551]], requires_grad=True), Parameter containing:
tensor([0.1892], requires_grad=True)]


In [22]:
model = MultivariateLinearRegressionModel()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.00001)  
## lr은 중요한 하이퍼 파라미터. 0.01, 0.001, 0.00001을 넣어보고 비교해봐라

for epoch in range(nb_epochs) : 
    # hypothesis 계산
    hypothesis = model(x_train)  
        
    # cost 계산 
    cost = torch.nn.functional.mse_loss(hypothesis, y_train)   # 파이토치에서 제공하는 평균 제곱 오차 함수
    
    
    # cost로 hypothesis 개선 
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {}  Cost : {:.6f}'.format(epoch+1, nb_epochs, hypothesis.squeeze().detach(),
                                                               cost.item()
                                                               ))

Epoch    1/100 hypothesis: tensor([-8.2666, -2.7712, -6.4352, -6.7662, -0.5484])  Cost : 31427.130859
Epoch    2/100 hypothesis: tensor([60.9637, 80.4374, 75.5522, 82.5160, 62.9186])  Cost : 9852.556641
Epoch    3/100 hypothesis: tensor([ 99.7235, 127.0226, 121.4540, 132.5018,  98.4512])  Cost : 3090.071289
Epoch    4/100 hypothesis: tensor([121.4240, 153.1038, 147.1528, 160.4871, 118.3443])  Cost : 970.390503
Epoch    5/100 hypothesis: tensor([133.5735, 167.7055, 161.5408, 176.1551, 129.4816])  Cost : 305.981445
Epoch    6/100 hypothesis: tensor([140.3759, 175.8803, 169.5961, 184.9271, 135.7167])  Cost : 97.723663
Epoch    7/100 hypothesis: tensor([144.1845, 180.4569, 174.1061, 189.8383, 139.2072])  Cost : 32.445244
Epoch    8/100 hypothesis: tensor([146.3171, 183.0190, 176.6312, 192.5879, 141.1612])  Cost : 11.982953
Epoch    9/100 hypothesis: tensor([147.5114, 184.4532, 178.0449, 194.1274, 142.2549])  Cost : 5.568195
Epoch   10/100 hypothesis: tensor([148.1802, 185.2560, 178.8365, 1